In [1]:
from openpyxl import load_workbook, Workbook
import numpy as np

In [3]:
def get_results(filenames, contrast, metric):
    col_dict = {'MSE': 1, 'MAE':2, 'SSIM':3, 'PSNR':4, "LPIPS":5}
    numbers = []
    for filename in filenames:
        wb = load_workbook(filename)
        ws = wb[contrast]
        column = col_dict[metric]
        rows = list(ws.rows)
        # skip the first row
        for row in rows[1:]:
            numbers.append(row[column].value)
    return np.mean(numbers), np.std(numbers)

In [4]:
def parse_scenario(scenario, contrast):
    output = []
    for s, contrast in zip(scenario, contrast):
        if s == '0':
            output.append(contrast)
    return output

In [22]:
dataset = 'IXI'
model = 'single'
scenarios = ['011', '101', '110']
contrasts = ['T1', 'T2', 'PD']
column_dict = {'T1': 3, 'T2': 4, 'PD': 5}
methods = ['MILR', 'MMGAN', 'MMT_new_0217', 'ResViT_28'] #, 'ResViT']
metrics = ['SSIM', 'PSNR', 'LPIPS']

for metric in metrics:
    wb = Workbook()
    ws = wb.active
    ws.cell(1, 1).value = 'Input'
    ws.cell(1, 2).value = 'Method'
    for i, contrast in enumerate(contrasts):
        ws.cell(1, 3+i).value = contrast
    current_row = 2
    for scenario in scenarios:
        output_contrasts = parse_scenario(scenario, contrasts)
        ws.cell(current_row, 1).value = scenario
        ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+len(methods)-1, end_column=1)
        for method in methods:
            fn = f'{dataset}/{model}/{method}/{scenario}.xlsx'
            for output_contrast in output_contrasts:
                mean, std = get_results([fn], output_contrast, metric)
                ws.cell(current_row, 2).value = method
                ws.cell(current_row, column_dict[output_contrast]).value = f'{mean:.4f}±{std:.4f}'
            current_row += 1
    wb.save(f"{dataset}_{model}_{metric}.xlsx")
                
        

In [21]:
dataset = 'BRATS'
model = 'single'
scenarios = ['0111', '1011', '1101', '1110']
contrasts = ['T1', 'T1Gd', 'T2', 'FLAIR']
column_dict = {'T1': 3, 'T1Gd': 4, 'T2': 5, 'FLAIR': 6}
methods = ['MILR', 'MMGAN', 'MMT_0.1'] #, 'ResViT']
metrics = ['SSIM', 'PSNR', 'LPIPS']

for metric in metrics:
    wb = Workbook()
    ws = wb.active
    ws.cell(1, 1).value = 'Input'
    ws.cell(1, 2).value = 'Method'
    for i, contrast in enumerate(contrasts):
        ws.cell(1, 3+i).value = contrast
    current_row = 2
    for scenario in scenarios:
        output_contrasts = parse_scenario(scenario, contrasts)
        ws.cell(current_row, 1).value = scenario
        ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+len(methods)-1, end_column=1)
        for method in methods:
            fn = f'{dataset}/{model}/{method}/{scenario}.xlsx'
            for output_contrast in output_contrasts:
                mean, std = get_results([fn], output_contrast, metric)
                ws.cell(current_row, 2).value = method
                ws.cell(current_row, column_dict[output_contrast]).value = f'{mean:.4f}±{std:.4f}'
            current_row += 1
    wb.save(f"{dataset}_{model}_{metric}.xlsx")
                
        

FileNotFoundError: [Errno 2] No such file or directory: 'BRATS/single/ResViT_28/0111.xlsx'

In [ ]:
dataset = 'IXI'
model = 'random'
scenarios = {'1': ['100', '010', '001'], '2': ['011', '101', '110']}

contrasts = ['T1', 'T2', 'PD']
column_dict = {'T1': 3, 'T2': 4, 'PD': 5}
#methods = ['MILR', 'MMGAN', 'MMT', 'MMT_new']
methods = ['MILR', 'MMGAN', 'MMT_new', 'ResViT_28'] #, 'ResViT']  # MMT_new is better
metrics = ['SSIM', 'PSNR', 'LPIPS']

for metric in metrics:
    wb = Workbook()
    ws = wb.active
    ws.cell(1, 1).value = 'Input'
    ws.cell(1, 2).value = 'Method'
    for i, contrast in enumerate(contrasts):
        ws.cell(1, 3+i).value = contrast
    current_row = 2
    for m, scenarios_m in scenarios.items():
        for scenario in scenarios_m:
            output_contrasts = parse_scenario(scenario, contrasts)
            ws.cell(current_row, 1).value = scenario
            ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+len(methods)-1, end_column=1)
            for method in methods:
                fn = f'{dataset}/{model}/{method}/{scenario}.xlsx'
                for output_contrast in output_contrasts:
                    mean, std = get_results([fn], output_contrast, metric)
                    ws.cell(current_row, 2).value = method
                    ws.cell(current_row, column_dict[output_contrast]).value = f'{mean:.4f}±{std:.4f}'
                current_row += 1
        # computer performance for m inputs
        ws.cell(current_row, 1).value = f'M={m}'
        ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+len(methods)-1, end_column=1)
        for method in methods:
            for output_contrast in contrasts:
                fns = []
                for scenario in scenarios_m:
                    bit = column_dict[output_contrast] - 3
                    if scenario[bit] == '0':  # include the scenario where output_contrast is '0'
                        fns.append(f'{dataset}/{model}/{method}/{scenario}.xlsx') 
                mean, std = get_results(fns, output_contrast, metric)
                ws.cell(current_row, 2).value = method
                ws.cell(current_row, column_dict[output_contrast]).value = f'{mean:.4f}±{std:.4f}'
            current_row += 1
    # computer overall performances
    ws.cell(current_row, 1).value = f'Overall'
    ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+len(methods)-1, end_column=1)
    all_scenarios = []
    for m, scenarios_m in scenarios.items():
        all_scenarios += scenarios_m
    for method in methods:
        for output_contrast in contrasts:
            fns = []
            for scenario in all_scenarios:
                bit = column_dict[output_contrast] - 3
                if scenario[bit] == '0':  # include the scenario where output_contrast is '0'
                    fns.append(f'{dataset}/{model}/{method}/{scenario}.xlsx') 
            mean, std = get_results(fns, output_contrast, metric)
            ws.cell(current_row, 2).value = method
            ws.cell(current_row, column_dict[output_contrast]).value = f'{mean:.4f}±{std:.4f}'
        current_row += 1
    wb.save(f"{dataset}_{model}_{metric}.xlsx")
                
        

In [18]:
dataset = 'BRATS'
model = 'random'
scenarios = {'1': ['1000', '0100', '0010', '0001'], '2': ['1100', '1010', '1001', '0110', '0101', '0011'], 
            '3': ['0111', '1011', '1101', '1110']}
contrasts = ['T1', 'T1Gd', 'T2', 'FLAIR']
column_dict = {'T1': 3, 'T1Gd': 4, 'T2': 5, 'FLAIR': 6}
#methods = ['MILR', 'MMGAN', 'MMT', 'MMT_0.01', 'MMT_0.05', 'MMT_0.075', 'MMT_0.1_0127', 'MMT_0.1_0202', 'MMT_0.1375', "MMT_0.15"]
methods = ['MILR', 'MMGAN', 'MMT_0.1_0202'] #, 'ResViT']
metrics = ['SSIM', 'PSNR', 'LPIPS']
for metric in metrics:
    wb = Workbook()
    ws = wb.active
    ws.cell(1, 1).value = 'Input'
    ws.cell(1, 2).value = 'Method'
    for i, contrast in enumerate(contrasts):
        ws.cell(1, 3+i).value = contrast
    current_row = 2
    for m, scenarios_m in scenarios.items():
        for scenario in scenarios_m:
            output_contrasts = parse_scenario(scenario, contrasts)
            ws.cell(current_row, 1).value = scenario
            ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+len(methods)-1, end_column=1)
            for method in methods:
                fn = f'{dataset}/{model}/{method}/{scenario}.xlsx'
                for output_contrast in output_contrasts:
                    mean, std = get_results([fn], output_contrast, metric)
                    ws.cell(current_row, 2).value = method
                    ws.cell(current_row, column_dict[output_contrast]).value = f'{mean:.4f}±{std:.4f}'
                current_row += 1
        # computer performance for m inputs
        ws.cell(current_row, 1).value = f'M={m}'
        ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+len(methods)-1, end_column=1)
        for method in methods:
            for output_contrast in contrasts:
                fns = []
                for scenario in scenarios_m:
                    bit = column_dict[output_contrast] - 3
                    if scenario[bit] == '0':  # include the scenario where output_contrast is '0'
                        fns.append(f'{dataset}/{model}/{method}/{scenario}.xlsx') 
                mean, std = get_results(fns, output_contrast, metric)
                ws.cell(current_row, 2).value = method
                ws.cell(current_row, column_dict[output_contrast]).value = f'{mean:.4f}±{std:.4f}'
            current_row += 1
    # computer overall performances
    ws.cell(current_row, 1).value = f'Overall'
    ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+len(methods)-1, end_column=1)
    all_scenarios = []
    for m, scenarios_m in scenarios.items():
        all_scenarios += scenarios_m
    for method in methods:
        for output_contrast in contrasts:
            fns = []
            for scenario in all_scenarios:
                bit = column_dict[output_contrast] - 3
                if scenario[bit] == '0':  # include the scenario where output_contrast is '0'
                    fns.append(f'{dataset}/{model}/{method}/{scenario}.xlsx') 
            mean, std = get_results(fns, output_contrast, metric)
            ws.cell(current_row, 2).value = method
            ws.cell(current_row, column_dict[output_contrast]).value = f'{mean:.4f}±{std:.4f}'
        current_row += 1
    wb.save(f"{dataset}_{model}_{metric}.xlsx")
                
        

In [27]:
ws[2][0].value

'BraTS2021_00111/050.npy'

In [37]:
ws[1][row_dict['MSE']].value

0.001654756478664013

In [62]:
a = 0.001654756478664013
f'{a:.4f}'

'0.0017'

In [38]:
metric = 'MSE'
numbers = []
for row in ws[1:]:
    numbers.append(row[row_dict[metric]].value)

In [77]:
bit

2

In [40]:
np.std(numbers)

0.001348306857626796